# 02 - Data Cleaning & Validation

This notebook cleans and validates the raw ESPN Soccer data, preparing it for feature engineering.

## Objectives
1. Filter to completed matches only
2. Remove duplicates and handle missing values
3. Merge fixtures with team stats, teams, and leagues
4. Apply league tier classification
5. Output clean `matches_base.parquet`

In [14]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 50)

# Paths
DATA_DIR = Path('../data')
BASE_DATA = DATA_DIR / 'base_data'
PROCESSED_DIR = DATA_DIR / 'processed'
PROCESSED_DIR.mkdir(exist_ok=True)

print("📂 Loading data...")

📂 Loading data...


## 1. Load Raw Data

In [15]:
# Load base tables
fixtures = pd.read_csv(BASE_DATA / 'fixtures.csv')
teams = pd.read_csv(BASE_DATA / 'teams.csv')
leagues = pd.read_csv(BASE_DATA / 'leagues.csv')
team_stats = pd.read_csv(BASE_DATA / 'teamStats.csv')
standings = pd.read_csv(BASE_DATA / 'standings.csv')
status = pd.read_csv(BASE_DATA / 'status.csv')

print(f"✅ Loaded fixtures: {len(fixtures):,} rows")
print(f"✅ Loaded teams: {len(teams):,} rows")
print(f"✅ Loaded leagues: {len(leagues):,} rows")
print(f"✅ Loaded team_stats: {len(team_stats):,} rows")
print(f"✅ Loaded standings: {len(standings):,} rows")

✅ Loaded fixtures: 67,353 rows
✅ Loaded teams: 4,144 rows
✅ Loaded leagues: 1,084 rows
✅ Loaded team_stats: 103,787 rows
✅ Loaded standings: 6,071 rows


## 2. Filter to Completed Matches

In [16]:
# Completed match status IDs
COMPLETED_STATUSES = [
    28,  # Full Time
    45,  # Final - After Extra Time
    46,  # Final - After Golden Goal
    47,  # Final - After Penalties
    51   # Final - Abandoned (but with result)
]

# Filter fixtures
matches = fixtures[fixtures['statusId'].isin(COMPLETED_STATUSES)].copy()
print(f"📊 Completed matches: {len(matches):,} ({len(matches)/len(fixtures)*100:.1f}% of total)")

# Check for duplicates
duplicates = matches['eventId'].duplicated().sum()
print(f"🔍 Duplicate eventIds: {duplicates}")
if duplicates > 0:
    matches = matches.drop_duplicates(subset='eventId', keep='first')
    print(f"   Removed {duplicates} duplicates")

📊 Completed matches: 57,870 (85.9% of total)
🔍 Duplicate eventIds: 0


## 3. Standardize Date Format

In [17]:
# Convert date to datetime
matches['date'] = pd.to_datetime(matches['date'], utc=True)
matches = matches.sort_values('date').reset_index(drop=True)

print(f"📅 Date range: {matches['date'].min()} to {matches['date'].max()}")
print(f"   Total days: {(matches['date'].max() - matches['date'].min()).days}")

📅 Date range: 2024-01-01 05:00:00+00:00 to 2025-12-15 20:45:00+00:00
   Total days: 714


## 4. League Tier Classification

In [18]:
# Define league tiers
TIER_1_MIDSIZE = ['ENG.1', 'ESP.1', 'ITA.1', 'GER.1', 'FRA.1']
TIER_2_MIDSIZE = ['NED.1', 'POR.1', 'BEL.1', 'TUR.1', 'SCO.1', 
                  'UEFA.CHAMPIONS', 'UEFA.EUROPA', 'ENG.2', 'ESP.2', 
                  'ITA.2', 'GER.2', 'FRA.2']

# Create league lookup with tier
league_lookup = leagues[['leagueId', 'midsizeName', 'leagueName', 'year']].drop_duplicates(subset='leagueId')

def assign_tier(midsize):
    if midsize in TIER_1_MIDSIZE:
        return 1
    elif midsize in TIER_2_MIDSIZE:
        return 2
    else:
        return 3

league_lookup['tier'] = league_lookup['midsizeName'].apply(assign_tier)

print("🏆 League Tier Distribution:")
print(league_lookup['tier'].value_counts().sort_index())

🏆 League Tier Distribution:
tier
1      5
2     12
3    203
Name: count, dtype: int64


## 5. Merge with Team Names

In [19]:
# Prepare team lookup
team_lookup = teams[['teamId', 'displayName', 'shortDisplayName']].copy()
team_lookup.columns = ['teamId', 'team_name', 'team_short_name']

# Merge home team
matches = matches.merge(
    team_lookup.rename(columns={'teamId': 'homeTeamId', 
                                'team_name': 'home_team_name',
                                'team_short_name': 'home_team_short'}),
    on='homeTeamId',
    how='left'
)

# Merge away team
matches = matches.merge(
    team_lookup.rename(columns={'teamId': 'awayTeamId',
                                'team_name': 'away_team_name',
                                'team_short_name': 'away_team_short'}),
    on='awayTeamId',
    how='left'
)

print(f"✅ Merged team names")
print(f"   Missing home team names: {matches['home_team_name'].isna().sum()}")
print(f"   Missing away team names: {matches['away_team_name'].isna().sum()}")

✅ Merged team names
   Missing home team names: 0
   Missing away team names: 0


## 6. Merge with League Info

In [20]:
# Merge league info
matches = matches.merge(
    league_lookup[['leagueId', 'midsizeName', 'leagueName', 'tier']].rename(
        columns={'midsizeName': 'league_code', 'leagueName': 'league_name'}
    ),
    on='leagueId',
    how='left'
)

print(f"✅ Merged league info")
print(f"\n📊 Matches by Tier:")
print(matches['tier'].value_counts().sort_index())

✅ Merged league info

📊 Matches by Tier:
tier
1.0     2472
2.0     5653
3.0    49237
Name: count, dtype: int64


## 7. Merge with Team Stats

In [21]:
# Key stats columns to keep
STATS_COLS = [
    'eventId', 'teamId', 'teamOrder',
    'possessionPct', 'foulsCommitted', 'yellowCards', 'redCards',
    'offsides', 'wonCorners', 'saves', 'totalShots', 'shotsOnTarget',
    'shotPct', 'accuratePasses', 'totalPasses', 'passPct'
]

stats_subset = team_stats[STATS_COLS].copy()

# Split into home (teamOrder=0) and away (teamOrder=1)
home_stats = stats_subset[stats_subset['teamOrder'] == 0].copy()
away_stats = stats_subset[stats_subset['teamOrder'] == 1].copy()

# Rename columns for home stats
home_stats_cols = {col: f'home_{col}' for col in STATS_COLS if col not in ['eventId', 'teamId', 'teamOrder']}
home_stats = home_stats.rename(columns=home_stats_cols)
home_stats = home_stats.drop(columns=['teamId', 'teamOrder'])

# Rename columns for away stats
away_stats_cols = {col: f'away_{col}' for col in STATS_COLS if col not in ['eventId', 'teamId', 'teamOrder']}
away_stats = away_stats.rename(columns=away_stats_cols)
away_stats = away_stats.drop(columns=['teamId', 'teamOrder'])

print(f"📊 Home stats records: {len(home_stats):,}")
print(f"📊 Away stats records: {len(away_stats):,}")

📊 Home stats records: 51,873
📊 Away stats records: 51,914


In [22]:
# Merge stats with matches
matches = matches.merge(home_stats, on='eventId', how='left')
matches = matches.merge(away_stats, on='eventId', how='left')

# Check coverage
stats_coverage = matches['home_possessionPct'].notna().sum() / len(matches) * 100
print(f"✅ Merged team stats")
print(f"   Stats coverage: {stats_coverage:.1f}%")

✅ Merged team stats
   Stats coverage: 48.8%


## 8. Handle Missing Values

In [23]:
# Fill missing numeric stats with 0 (conservative approach)
stats_columns = [col for col in matches.columns if col.startswith(('home_', 'away_')) 
                 and any(s in col for s in ['Pct', 'Shots', 'Corners', 'Cards', 'Passes', 'fouls', 'offsides', 'saves'])]

print(f"📊 Filling {len(stats_columns)} stats columns with 0 for missing values")

for col in stats_columns:
    missing_before = matches[col].isna().sum()
    if missing_before > 0:
        matches[col] = matches[col].fillna(0)

# Verify
remaining_nulls = matches[stats_columns].isna().sum().sum()
print(f"✅ Remaining nulls in stats columns: {remaining_nulls}")

📊 Filling 24 stats columns with 0 for missing values
✅ Remaining nulls in stats columns: 0


## 9. Data Validation

In [24]:
# Validate data integrity
print("🔍 Data Validation")
print("=" * 50)

# Check score ranges
max_home_score = matches['homeTeamScore'].max()
max_away_score = matches['awayTeamScore'].max()
print(f"   Max home score: {max_home_score}")
print(f"   Max away score: {max_away_score}")

# Flag anomalous scores (>10 goals)
anomalous = matches[(matches['homeTeamScore'] > 10) | (matches['awayTeamScore'] > 10)]
print(f"   Matches with score > 10: {len(anomalous)}")

# Check possession ranges
if 'home_possessionPct' in matches.columns:
    invalid_possession = matches[
        (matches['home_possessionPct'] > 100) | 
        (matches['away_possessionPct'] > 100)
    ]
    print(f"   Invalid possession (>100%): {len(invalid_possession)}")

print("\n✅ Validation complete")

🔍 Data Validation
   Max home score: 16
   Max away score: 19
   Matches with score > 10: 29
   Invalid possession (>100%): 0

✅ Validation complete


## 10. Select Final Columns

In [25]:
# Define final column order
FINAL_COLUMNS = [
    # Match identifiers
    'eventId', 'date', 'leagueId', 'league_code', 'league_name', 'tier',
    'venueId', 'attendance',
    
    # Teams
    'homeTeamId', 'home_team_name', 'home_team_short',
    'awayTeamId', 'away_team_name', 'away_team_short',
    
    # Scores
    'homeTeamScore', 'awayTeamScore',
    'homeTeamWinner', 'awayTeamWinner',
    
    # Home team stats
    'home_possessionPct', 'home_totalShots', 'home_shotsOnTarget',
    'home_wonCorners', 'home_foulsCommitted', 'home_yellowCards', 'home_redCards',
    'home_accuratePasses', 'home_totalPasses', 'home_passPct',
    
    # Away team stats
    'away_possessionPct', 'away_totalShots', 'away_shotsOnTarget',
    'away_wonCorners', 'away_foulsCommitted', 'away_yellowCards', 'away_redCards',
    'away_accuratePasses', 'away_totalPasses', 'away_passPct',
    
    # Status
    'statusId'
]

# Select columns that exist
available_cols = [col for col in FINAL_COLUMNS if col in matches.columns]
matches_clean = matches[available_cols].copy()

print(f"📋 Final columns: {len(available_cols)}")
print(f"📊 Final rows: {len(matches_clean):,}")

📋 Final columns: 39
📊 Final rows: 59,550


## 11. Export Clean Data

In [26]:
# Save to parquet
output_path = PROCESSED_DIR / 'matches_base.parquet'
matches_clean.to_parquet(output_path, index=False)

print(f"✅ Saved to {output_path}")
print(f"   File size: {output_path.stat().st_size / 1024 / 1024:.2f} MB")

# Also save league lookup
league_output = PROCESSED_DIR / 'leagues_clean.parquet'
league_lookup.to_parquet(league_output, index=False)
print(f"✅ Saved leagues to {league_output}")

✅ Saved to ../data/processed/matches_base.parquet
   File size: 2.39 MB
✅ Saved leagues to ../data/processed/leagues_clean.parquet


In [27]:
# Summary
print("\n" + "=" * 50)
print("📊 CLEANING SUMMARY")
print("=" * 50)
print(f"   Input fixtures: {len(fixtures):,}")
print(f"   Output matches: {len(matches_clean):,}")
print(f"   Tier 1 matches: {len(matches_clean[matches_clean['tier'] == 1]):,}")
print(f"   Tier 2 matches: {len(matches_clean[matches_clean['tier'] == 2]):,}")
print(f"   Tier 3 matches: {len(matches_clean[matches_clean['tier'] == 3]):,}")
print(f"\n   Stats coverage: {(matches_clean['home_possessionPct'] > 0).sum() / len(matches_clean) * 100:.1f}%")


📊 CLEANING SUMMARY
   Input fixtures: 67,353
   Output matches: 59,550
   Tier 1 matches: 2,472
   Tier 2 matches: 5,798
   Tier 3 matches: 50,736

   Stats coverage: 48.6%


## Next: 03_feature_engineering.ipynb

The clean `matches_base.parquet` is ready for feature engineering:
- Rolling form calculations (last 5/10 games)
- Home/away specific metrics
- Head-to-head history
- Pre-match standings context